# Getting Started



---

### 1. Setup

**Step 1: Gain Access to the Model**: If you have not yet requested for model access in Bedrock, you do so [request access following these instructions](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html).

In [ ]:
%pip install boto3

### 2. Text Understanding [Applicable for Nova Micro as well as Lite]

The example below demonstrates how to use a text-based prompt, including a simple system prompt and message list.


Note: Below examples are using Nova Lite for Illustrative Purposes but you can make use of Micro Models for Text Understanidng Usecases as well.


In [ ]:
import boto3
import json
import base64
import datetime

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-east-1")



#### 2.1 Synchronous API Call

In [ ]:
# Define your system prompt(s).
system_list = [
    { "text": "You should respond to all messages in french" }
]

# Define one or more messages using the "user" and "assistant" roles.
message_list = [
    {"role": "user", "content": [{"text": "tell me a joke"}]},
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
request_id = response["ResponseMetadata"]["RequestId"]
print(f"Request ID: {request_id}")
model_response = json.loads(response["body"].read())

# Pretty print the response JSON.
print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)

#### 2.2 Streaming API Call

The example below demonstrates how to use a text-based prompt with the streaming API.

In [ ]:

# Define your system prompt(s).
system_list = [
    { "text": "Act as a creative writing assistant. When the user provides you with a topic, write a short story about that topic." }
]

# Define one or more messages using the "user" and "assistant" roles.
message_list = [{"role": "user", "content": [{"text": "A camping trip"}]}]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 500, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

request_body = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

start_time = datetime.now()

# Invoke the model with the response stream
response = client.invoke_model_with_response_stream(
    modelId=LITE_MODEL_ID, body=json.dumps(request_body)
)

request_id = response.get("ResponseMetadata").get("RequestId")
print(f"Request ID: {request_id}")
print("Awaiting first token...")

chunk_count = 0
time_to_first_token = None

# Process the response stream
stream = response.get("body")
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            # Print the response chunk
            chunk_json = json.loads(chunk.get("bytes").decode())
            # Pretty print JSON
            # print(json.dumps(chunk_json, indent=2, ensure_ascii=False))
            content_block_delta = chunk_json.get("contentBlockDelta")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = datetime.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")

                chunk_count += 1
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S:%f")
                # print(f"{current_time} - ", end="")
                print(content_block_delta.get("delta").get("text"), end="")
    print(f"Total chunks: {chunk_count}")
else:
    print("No response stream received.")

### 3. Multimodal Understanding [Applicable only for Nova lite Model]

The following examples show how to pass various media types to the model. *(reminder - this is only supported with the Lite model)*

#### 3.1 Image Understanding

Lets see how Nova model does on Image Understanding Usecase. 

Here we will pass an Image of a Sunset and ask model to try to create 3 art titles for this image. 

![A Sunset Image](media/sunset.png)

In [ ]:


# Open the image you'd like to use and encode it as a Base64 string.
with open("media/sunset.png", "rb") as image_file:
    binary_data = image_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode("utf-8")

# Define your system prompt(s).
system_list = [
    { "text": "You are an expert artist. When the user provides you with an image, provide 3 potential art titles" }
]

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "png",
                    "source": {"bytes": base64_string},
                }
            },
            {"text": "Provide art titles for this image."},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20, "temperature": 0.3}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# Pretty print the response JSON.
print("[Full Response]")
print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)

#### 3.2 Video Understanding

Lets now, try to see how Nova does on Video understanding use case

Here we are going to pass in a video with Quesdilla making instructions.


In [ ]:
from IPython.display import Video

Video("media/cooking-quesadilla.mp4")

In [ ]:

# Open the image you'd like to use and encode it as a Base64 string.
with open("media/cooking-quesadilla.mp4", "rb") as video_file:
    binary_data = video_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode("utf-8")

# Define your system prompt(s).
system_list = [
    { "text": "You are an expert media analyst. When the user provides you with a video, provide 3 potential video titles" }
]

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": "mp4",
                    "source": {"bytes": base64_string},
                }
            },
            {"text": "Provide video titles for this clip."},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20, "temperature": 0.3}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# Pretty print the response JSON.
print("[Full Response]")
print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)

### Video Understanding using S3 Path

In [ ]:

# Define your system prompt(s).
system_list = [
    { "text": "You are an expert media analyst. When the user provides you with a video, provide 3 potential video titles" }
]

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": "mp4",
                    "source": {
                        "s3Location": {
                            "uri": "s3://905418197933-demo-bucket/cooking-quesadilla.mp4"
                        }
                    },
                }
            },
            {"text": "Provide video titles for this clip."},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20, "temperature": 0.3}

native_request = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

# Invoke the model and extract the response body.
response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# Pretty print the response JSON.
print("[Full Response]")
print(json.dumps(model_response, indent=2))

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)